In [41]:

# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [42]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [43]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
         return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [44]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [45]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [46]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [47]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [48]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [49]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nanonets/Nanonets-OCR-s',
 '/MiniMaxAI/MiniMax-M1-80k',
 '/tencent/Hunyuan3D-2.1',
 '/echo840/MonkeyOCR',
 '/Menlo/Jan-nano',
 '/models',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/enzostvs/deepsite',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/MiniMaxAI/MiniMax-M1',
 '/spaces/nvidia/PartPacker',
 '/spaces',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Nemotron-Personas',
 '/datasets/lingshu-medical-mllm/ReasonMed',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',
 '

In [50]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

In [51]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [52]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nanonets/Nanonets-OCR-s
Updated
4 days ago
•
28.4k
•
869
MiniMaxAI/MiniMax-M1-80k
Updated
1 day ago

In [53]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [60]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
  
    return user_prompt

In [61]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnanonets/Nanonets-OCR-s\nUpdated\n4 days ago\n•\n28.4k\n•\n869\nMiniMaxAI/MiniMax-M1-80k\nUpdated\n1 day ago\n•\n1.8k\n•\n404\ntencent/Hunyuan3D-2.1\nUpdated\nabout 18 hours ago\n•\n7.77k\n•\n369\necho840/MonkeyOCR\nUpdated\nabout 12 hours ago\n•\n4\n•\n407\nMenlo/Jan-nano\nUpdated\n2 days ago\n•\n4.96k\n•\n274\nBrowse 1M+ models\nSpaces\nRunning\n465\n465\nSparc3D\n🏃\nNext-Gen High-Resolu

In [62]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [63]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is a leading platform at the forefront of the AI community, dedicated to building the future of machine learning (ML). We provide a collaborative space for developers, researchers, and organizations to discover, create, and share models, datasets, and applications. With a rich collection of over **1 million models** and **250,000 datasets**, we are committed to democratizing access to machine learning tools and fostering innovation.

## Our Offerings
### **Models and Datasets**
- **Explore AI Models:** Browse and utilize cutting-edge models across various domains including text, image, video, audio, and even 3D.
- **Datasets:** Access a wide array of datasets that are continually updated and maintained by our community.
- **Spaces:** Run and collaborate on applications seamlessly, making ML development more efficient and engaging.

### **Enterprise Solutions**
For organizations seeking advanced capabilities, Hugging Face offers:
- **Compute Services** that allow you to deploy optimized inference environments.
- **Enterprise Packages** providing dedicated support, security, and control for your team.

## Our Community
With over **50,000 organizations** using our platform, we pride ourselves on being a vibrant ecosystem filled with scholars, technologists, and AI enthusiasts. Our community collaborates on projects and shares knowledge, driving advancements in machine learning practices. Among our notable users are industry giants such as **Google, Microsoft, Amazon, and Meta**.

## Company Culture
At Hugging Face, we embrace a culture that values:
- **Collaboration:** We believe the best results are achieved through teamwork and open communication.
- **Innovation:** We encourage creativity and experimentation, supporting our members in their quest to push the boundaries of AI.
- **Inclusivity:** Our community is open to everyone, from newcomers to experts, fostering diversity in thought and experience.

## Careers
Join our team and help shape the future of AI! We are always looking for talented individuals who are passionate about machine learning. Be part of a cutting-edge mission to democratize technology and foster an inclusive community.

### Current Opportunities
Explore career openings in various domains including:
- Research
- Engineering
- Community Management
  
For more information on current job openings, check our [Careers Page](https://huggingface.co/jobs).

## Get In Touch
Interested in becoming a part of our journey? 
- **Website:** [Hugging Face](https://huggingface.co)
- **Social Media:** Follow us on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), and join our community on [Discord](https://discord.gg/huggingface).

---

Hugging Face is where machine learning enthusiasts come together to build, innovate, and share, all while contributing to a brighter future with AI. Join us!

In [64]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [65]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


```markdown
# Hugging Face Company Brochure

## Overview
**Hugging Face** is an innovative platform at the forefront of the artificial intelligence community. With a mission to democratize machine learning (ML), Hugging Face serves as a collaborative hub where machine learning enthusiasts and professionals can work together on models, datasets, and applications. By fostering a strong community spirit, Hugging Face is building the future of AI.

## Our Offerings
- **Models**: Access over **1 million models** spanning various capabilities, including text, image, video, audio, and even 3D applications.
- **Datasets**: Explore and share over **250,000 datasets** meticulously curated for a variety of ML tasks.
- **Spaces**: Host and showcase applications easily, with over **400,000 applications** currently running on the platform.
- **Compute and Enterprise Solutions**: Tailored services including optimized inference endpoints starting at just **$0.60/hour** for GPU access, and enterprise-grade solutions that emphasize security and dedicated support.

## Our Customers
More than **50,000 organizations** utilize Hugging Face, including industry giants such as:
- **Meta**
- **Google**
- **Microsoft**
- **Amazon**
Hugging Face is the choice for organizations looking to harness the power of AI through collaboration, innovation, and state-of-the-art technology.

## Community & Culture
At Hugging Face, community is at the heart of everything we do. We celebrate collaboration through:
- Regular updates and contributions from users, enhancing our datasets and models.
- A vibrant community forum and active user involvement in discussions and contributions.
- Commitment to open-source values, continually evolving our tooling to meet the needs of ML practitioners.

Our culture encourages creativity, diversity, and shared learning, making it a dynamic environment for anyone passionate about AI and machine learning.

## Careers at Hugging Face
Hugging Face is always on the lookout for talented individuals who want to be a part of an exciting journey in AI. We offer:
- **Opportunities** in various roles across engineering, research, product management, and sales.
- An inclusive environment focused on personal and professional growth.
- The chance to work at a company that values innovation and collaboration above all.

Join us in our mission to shape the future of AI! Check our careers page for current job openings.

---

**Connect With Us**
- [Website](https://huggingface.co)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)

For more information or to get involved, reach out to us or sign up on our platform today. Let’s build the future together!
```